<h1>Casestudy Eric Bühler - Aachen </h1>

In [11]:
# Packages used in this notebook
import requests
from bs4 import BeautifulSoup
import collections
from datetime import datetime, timedelta
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
import numpy as np
import pandas as pd
import configparser
import openai


In [12]:
def open_AI_call():
    # Your OpenAI API key
        api_key = 'key'

        # Initialize the OpenAI client
        openai.api_key = api_key

        # Define the messages to send to the API based on your previous message
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Welche PLZ hat dieser Stadtteil von Aachen? f{stadtteil} Bitte schreib nur die Zahl"}
        ]

        # Call the OpenAI API with the chat completion method
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",  # Specify the model to use, you can change this as needed
            messages=messages,
            max_tokens=10,  # Adjust the number of tokens as needed
            n=1,
            stop=None,
            temperature=0.7
        )

        # Extract the text response
        plz = response.choices[0].message['content'].str

In [13]:


def scrape_wgs(pages=2):
    anzeigen=[]
    for i in range(pages):
        url = f"https://www.wg-gesucht.de/wg-zimmer-in-Aachen.1.0.1.{i}.html?offer_filter=1&city_id=1&sort_order=0&noDeact=1&categories%5B%5D=0&pagination=1&pu="
        # Sends a GET request to the URL to retrieve its content
        response = requests.get(url)
        assert response.status_code == 200
        #print(response.text)
        # Parse results
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extracting all Anzeigen
        anzeigen_raw = soup.find_all('div', class_='col-sm-8 card_body')
        # Loop through each anzeige section and build dictionaries of them
        raw_text = ''
        for anzeige in anzeigen_raw:
            titel = anzeige.find('a').get_text().replace('\n', '').strip()
            preis = anzeige.find('div', class_='col-xs-3').get_text().replace('€','').strip()
            frei_ab = anzeige.find('div', class_='col-xs-5 text-center').get_text().replace('\n', '').split('-')[0].strip()
            qm = anzeige.find('div', class_='col-xs-3 text-right').get_text().replace('\n', '').replace('m²','').strip()
            wg_groesse = anzeige.find('div', class_='col-xs-11').get_text().split('er')[0].split()[0]
            stadtteil = anzeige.find('div', class_='col-xs-11').get_text().strip().split('|')[1].replace('\n', '').strip().split()
            #uncomment if i get rich
            #plz = open_AI_call()
            
            online_seit = anzeige.find('div', class_='col-xs-9').get_text().split('Online:')[1].strip()
            if "Sekunde" in online_seit or "Sekunden" in online_seit:
                online_seit="00:00"

            if "Minuten" in online_seit or "Minute" in online_seit:
                minutes = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"00:{minutes.zfill(2)}"  # Ensuring minutes are two digits

            if "Stunden" in online_seit or "Stunde" in online_seit:
                hours = ''.join(filter(str.isdigit, online_seit))
                online_seit = f"{hours.zfill(2)}:00"  # Ensuring hours are two digits

            if "Tag" in online_seit or "Tage" in online_seit:
                days = ''.join(filter(str.isdigit, online_seit))
                hours = int(days)*24
                online_seit = f"{hours}:00"

            
            anzeigen.append({'titel': titel,'preis' : preis,'frei_ab':frei_ab, 'qm': qm,'wg_groesse':wg_groesse,'stadtteil':stadtteil,'online_seit':online_seit})
    return pd.DataFrame(anzeigen)

# Display the DataFrame
#test_data= scrape_wgs(10)
display(test_data)

,titel,preis,frei_ab,qm,wg_groesse,stadtteil,online_seit
0,Studentenzimmer in direkter Uninähe,265,01.09.2024,20,11,"[Aachen, Aachen]",00:08
1,Zimmer in bester Studentenverbindung,260,12.06.2024,15,17,"[Aachen, Aachen]",00:09
2,Zimmer in 2-Personen WG,417,09.06.2024,20,2,"[Aachen, Aachen]",00:10
3,2 schöne möblierte Zimmer in Aachen-Mitte mit ...,279,09.05.2024,16,14,"[Aachen, Aachen]",00:13
4,WG-Zimmer in bester Lage für Studenten,200,01.07.2024,14,6,"[Aachen, Aachen]",00:13
...,...,...,...,...,...,...,...
175,16m^2 Zimmer in top Lage,325,15.08.2024,16,4,"[Aachen, Aachen]",96:00
176,WG-Zimmer in zentraler Lage,470,01.07.2024,15,3,"[Aachen, Mitte]",96:00
177,Möbliertes WG Zimmer in ansprechender Doppelha...,500,07.06.2024,14,4,"[Aachen, Steppenberg]",96:00
178,Zimmer in bester Lage in der vierer WG mit den...,400,07.06.2024,12,4,"[Aachen, Aachen-Mitte]",96:00
